<a href="https://colab.research.google.com/github/Yang-Hyun-Jun/Competition/blob/main/S1_T1_gap_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import

In [ ]:
!pip install kora -q
import kora.install.rdkit

     |████████████████████████████████| 57 kB 1.5 MB/s 
     |████████████████████████████████| 56 kB 2.7 MB/s 


In [ ]:
!pip install lightgbm

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Reshape
from tensorflow.keras.layers import BatchNormalization, Activation, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from rdkit import Chem

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/samsung/"

### 전처리 

In [ ]:
train_calcdesc = pd.read_csv(path + "train_calcdesc.csv")
train_calcdesc

,uid,SMILES,S1_energy(eV),T1_energy(eV),abonds,atoms,bonds,dbonds,HBA1,HBA2,HBD,logP,MP,MR,MW,nF,rotors,sbonds,tbonds,TPSA,gap
0,train_0,CCC1CCCCN1C(=O)C(C)OC(=O)c1c(C)oc(-n2cccc2)c1C#N,4.6747,3.3809,10.0,53.0,55.0,2.0,30.0,7.0,0.0,3.52468,148.1122,107.1555,383.44090,0.0,7.0,42.0,1.0,88.47,1.2938
1,train_1,COc1ccc(Oc2ccc(N3C(=S)NC(c4ccccn4)C3c3cc(C)n(-...,3.6617,3.4585,29.0,71.0,76.0,1.0,36.0,8.0,2.0,8.21990,315.7129,176.8537,597.12636,0.0,7.0,46.0,0.0,103.87,0.2032
2,train_2,CC(=O)Nc1ccc(C(=O)[C@H](C)Sc2nnc(C3CCCCC3)o2)cc1,3.6420,3.1787,11.0,49.0,51.0,2.0,29.0,7.0,1.0,4.51220,187.5404,101.7412,373.46922,0.0,7.0,38.0,0.0,110.39,0.4633
3,train_3,OC(CNC1CC1)CN1CCc2sccc2C1,4.8901,3.7847,5.0,37.0,39.0,0.0,23.0,4.0,2.0,1.54790,100.1604,74.4465,252.37570,0.0,5.0,34.0,0.0,63.74,1.1054
4,train_4,CCNC(CCCC(F)(F)F)C1(OCC)CCOCC1,6.4967,6.2724,0.0,46.0,46.0,0.0,29.0,3.0,1.0,3.67370,10.2322,72.4987,297.35695,3.0,9.0,46.0,0.0,30.49,0.2243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30265,train_30269,COc1c(/C=N/Nc2ccc(C#N)cc2)ccc2ccccc12,3.7109,2.4774,17.0,38.0,40.0,1.0,19.0,4.0,1.0,4.23908,176.3131,92.6327,301.34190,0.0,4.0,21.0,1.0,57.41,1.2335
30266,train_30270,Cc1cc(=O)oc2c3c4c(cc12)CCCN4CCC3,3.8752,2.6624,11.0,36.0,39.0,1.0,19.0,3.0,0.0,2.86520,151.0189,79.9100,255.31168,0.0,0.0,27.0,0.0,33.45,1.2128
30267,train_30271,Cc1cc(=O)oc2ccc3ccc(-c4cn(Cc5ccccc5)nn4)cc3c12,3.7905,2.6050,27.0,45.0,49.0,1.0,20.0,5.0,0.0,4.56140,235.3467,109.7210,367.39998,0.0,3.0,21.0,0.0,60.92,1.1855
30268,train_30272,CCCCCCN1C(=O)c2cccc3c(-c4cccs4)ccc(c23)C1=O,3.5176,2.4248,15.0,47.0,50.0,4.0,22.0,4.0,0.0,4.16050,194.6409,112.3385,363.47264,0.0,5.0,31.0,0.0,77.15,1.0928


In [ ]:
train_calcdesc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30270 entries, 0 to 30269
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uid            30270 non-null  object 
 1   SMILES         30270 non-null  object 
 2   S1_energy(eV)  30270 non-null  float64
 3   T1_energy(eV)  30270 non-null  float64
 4   abonds         30270 non-null  float64
 5   atoms          30270 non-null  float64
 6   bonds          30270 non-null  float64
 7   dbonds         30270 non-null  float64
 8   HBA1           30270 non-null  float64
 9   HBA2           30270 non-null  float64
 10  HBD            30270 non-null  float64
 11  logP           30270 non-null  float64
 12  MP             30270 non-null  float64
 13  MR             30270 non-null  float64
 14  MW             30270 non-null  float64
 15  nF             30270 non-null  float64
 16  rotors         30270 non-null  float64
 17  sbonds         30270 non-null  float64
 18  tbonds

In [ ]:
def fingerprint(df):
  fg_list = []
  smiles = df["SMILES"]
  
  for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    fg = Chem.RDKFingerprint(mol)
    fg_list.append(fg)
  
  fg_list = np.array(fg_list)
  return fg_list

def scaler(data):
  for i in range(data.shape[1]):
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    
    if max == min:
      data[:,i] = 0
    else:
      data[:,i] = (data[:,i]-min)/(max-min)              
  return data
  
def gen_xy(df, feat = False):
  features = df.iloc[:,np.arange(16)+4]
  features = features.to_numpy()
  features = scaler(features)
  
  if feat == False:
    x = fg_list.copy()
  if feat == True:
    x = np.concatenate([fg_list, features], axis=1)
    
  y = np.array(df["gap"])
  return x, y

In [ ]:
#분자 구조 SMILES 형식 변환
fg_list = fingerprint(train_calcdesc).astype(np.float64)
fg_list.shape

(30270, 2048)

In [ ]:
x_data, y_data = gen_xy(train_calcdesc)
x_data.shape, y_data.shape

((30270, 2048), (30270,))

### Models 

#### 개별 모델

In [ ]:
seed = 42

In [ ]:
### XGB 
import xgboost
n_est1 = 1000

xgb_parameter = {"objective":'reg:squarederror',
                 "subsample":0.6,
                 "colsample_bytree":0.7,
                 "learning_rate":0.05,
                 "random_state":seed,
                 "n_estimators":n_est1,
                 "max_depth":8,
                 "max_bin":260,
                 "max_leaves":5,
                 "num_iterations":1000,
                 "gamma":0.5}

xgb_model = xgboost.XGBRegressor(**xgb_parameter)

In [ ]:
### RandomForest
from sklearn.ensemble import RandomForestRegressor
n_est2 = 1000

RF_parameter = {'max_depth': 20,
                'min_samples_leaf': 2,
                'min_samples_split':6,
                'random_state': seed,
                'max_leaf_nodes': 500,
                'n_estimators': n_est2
                }

RF_model = RandomForestRegressor(**RF_parameter)

In [ ]:
### LGB
from lightgbm import LGBMRegressor
n_est3=2000

lgb_parameters = {'n_estimators': n_est3,
                  'random_state': seed,
                  'learning_rate': 0.03,
                  'num_leaves': 255,
                  'max_depth': 30,
                  'max_bin': 300,
                  'subsample':0.8,
                  'subsamle_freq':3,
                  'min_child_samples':2,
                  'colsample_bytree':0.6
                  }

lgb_model = LGBMRegressor(**lgb_parameters)

#### GridSearch

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

xgb_model = xgboost.XGBRegressor()
xgb_parameter = {"subsample":[0.5, 0.6, 0.7],
                 "colsample_bytree":[0.5, 0.6, 0.7],
                 "learning_rate":[0.05, 0.02, 0.01],
                 "random_state":[42],
                 "n_estimators":[2000, 3000, 5000],
                 "max_depth":[6, 8, 10],
                 "max_bin":[260, 255, 270],
                 "max_leaves":[5, 4, 3],
                 "num_iterations":[1000, 1100, 1200],
                 "gamma":[0.5, 0.4, 0.3]}

cv = KFold(n_splits=6, random_state=42)
GS=GridSearchCV(xgb_model, param_grid=xgb_parameter, cv=cv, scoring='f1', n_jobs=-1)

GS.fit(x_data, y_data)
print('final params', GS.best_params_)  
print('best score', GS.best_score_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

RF_model = RandomForestRegressor()
RF_parameter = {"max_depth":[8, 12, 20],
                "min_samples_leaf":[2, 4],
                "min_samples_split":[6, 8],
                "max_leaf_nodes":[1000, 2000, 3000],
                "n_estimators":[1000, 2000, 3000],
                "random_state":[42],
                # "criterion"
                }

cv = KFold(n_splits=6, random_state=42)
GS=GridSearchCV(RF_model, param_grid=RF_parameter, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)

GS.fit(x_data[:15000], y_data[:15000])
print('final params', GS.best_params_)  
print('best score', GS.best_score_)

#### Staking

In [ ]:
seed = 42 
n_folds = 6

In [ ]:
from sklearn.model_selection import KFold

def staking_dataset(model, x_train, y_train, x_test, n_folds):
  kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
  kf_x = kf.split(x_train)

  train_pred = np.zeros((x_train.shape[0], 1))
  test_pred = np.zeros((x_test.shape[0], n_folds))

  for i, (train_index, valid_index) in enumerate(kf_x):
    print(f"{i} fold set")
    x_tr = x_train[train_index]
    y_tr = y_train[train_index]
    x_va = x_train[valid_index]

    if model == fc_model:
      model.compile(Adam(3e-4), "MAE")
      model.fit(x_tr, y_tr, epochs=30) 
    else:
      model.fit(x_tr, y_tr)

    train_pred[valid_index,:] = model.predict(x_va).reshape((-1, 1))
    test_pred[:,i:i+1] = model.predict(x_test).reshape((-1, 1))

  test_pred_mean = np.mean(test_pred, axis=1).reshape((-1, 1))
  return train_pred, test_pred_mean

In [ ]:
fc_train, fc_test = staking_dataset(fc_model, x_data, y_data, fg_test, n_folds)
xgb_train, xgb_test = staking_dataset(xgb_model, x_data, y_data, fg_test, n_folds)
lgb_train, lgb_test = staking_dataset(lgb_model, x_data, y_data, fg_test, n_folds)

print(fc_train.shape, fc_test.shape)
print(xgb_train.shape, xgb_test.shape)
print(lgb_train.shape, lgb_test.shape)

0 fold set
Epoch 1/30
789/789 [==============================] - 11s 10ms/step - loss: 0.2804
Epoch 2/30
789/789 [==============================] - 7s 9ms/step - loss: 0.2059
Epoch 3/30
789/789 [==============================] - 7s 9ms/step - loss: 0.1840
Epoch 4/30
789/789 [==============================] - 8s 10ms/step - loss: 0.1703
Epoch 5/30
789/789 [==============================] - 7s 9ms/step - loss: 0.1586
Epoch 6/30
789/789 [==============================] - 8s 10ms/step - loss: 0.1492
Epoch 7/30
789/789 [==============================] - 7s 9ms/step - loss: 0.1420
Epoch 8/30
789/789 [==============================] - 7s 9ms/step - loss: 0.1344
Epoch 9/30
789/789 [==============================] - 8s 10ms/step - loss: 0.1297
Epoch 10/30
789/789 [==============================] - 8s 10ms/step - loss: 0.1235
Epoch 11/30
789/789 [==============================] - 7s 9ms/step - loss: 0.1169
Epoch 12/30
789/789 [==============================] - 8s 10ms/step - loss: 0.1149
Epoch 1

In [ ]:
stack_train = np.concatenate([fc_train, xgb_train, lgb_train], axis=1)
stack_test = np.concatenate([fc_test, xgb_test, lgb_test], axis=1)

stack_train.shape, stack_test.shape

In [ ]:
def D_block(x, filters, initializer=None):
  x = Dense(filters, kernel_initializer=initializer)(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  return x

def stacking_model():
  inputs = Input(2)

  d1 = D_block(inputs, 128, "normal")
  d2 = D_block(d1, 64)
  d3 = D_block(d2, 32)
  d4 = D_block(d3, 16)
  d5 = D_block(d3, 8)

  outputs = Dense(1, kernel_initializer="normal")(d4)

  model = Model(inputs, outputs)
  return model

stacking_model = stacking_model()
stacking_model.summary()

In [ ]:
stacking_model = xgboost.XGBRegressor(max_depth=6, n_estimators=500)
stacking_model.fit(stack_train, y_data, epochs=13)

### 추론

In [ ]:
test = pd.read_csv(path + "test.csv")
test

,uid,SMILES
0,test_0,COc1ccc(S(=O)(=O)NC2CCN(C3CCCCC3)CC2)c(C)c1C
1,test_1,CC(CCCC1CCC2C3=C(CC[C@]12C)[C@@]1(C)CC[C@H](C)...
2,test_2,C[C@@H]1C[C@@H]1c1ccc2c(c1)c(-c1ccc[nH]c1=O)c(...
3,test_3,CCCn1c(=O)c2ccccc2n2c(SCC(=O)NC(Cc3ccccc3)c3cc...
4,test_4,CC(C)CN(C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)OCc1cn...
...,...,...
597,test_597,N#Cc1c(-n2c3ccccc3c3ccccc32)c(-n2c3ccccc3c3ccc...
598,test_598,CC1(C)c2ccccc2N(c2ccc(-c3cc(-c4ccc(N5c6ccccc6C...
599,test_599,Cc1nc(-c2ccc(N3c4ccccc4C(C)(C)c4ccccc43)cc2)cc...
600,test_600,c1ccc2c(c1)Oc1ccccc1N2c1ccc(-c2nc3ccccc3s2)cc1


In [ ]:
fg_test = fingerprint(test)
fg_test.shape 

(602, 2048)

In [ ]:
stacking_pred = stacking_model.predict(stack_test)
stacking_pred.shape

(602,)